# Download file(.zip) from G-Drive Public Link

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Extracting zip file from google drive
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/My Drive/Data/Kiwi.zip", 'r')
zip_ref.extractall('/content/Dataset/')
zip_ref.close()
print("Extracted Sucessfully")

# Set Parameters

In [ ]:
path = '/content/Dataset/WS_Dataset/'
!ls '/content/Dataset/WS_Dataset/'
train_dir = path + 'train'
test_dir = path + 'test'
val_dir = path + 'val'
img_height = 256
img_width = 256
batch_size = 50
no_of_classes = 3
print("Parameter Set Done!! ")

In [ ]:
import os, os.path
def cal_step(dir, batch_size=batch_size):
  id = []
  for r,d,file in os.walk(dir):
      id.append(file)
  id = id[1:]
  sum = 0
  for i in id:
    sum += len(i)
  batch = int(sum/batch_size)
  return batch 

print("Test Step {}".format(cal_step(test_dir)))
print("Train Step {}".format(cal_step(train_dir)))
print("Val Step {}".format(cal_step(val_dir)))

# Pre-processing Fuctions

## Run ImageDataGenerator

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from skimage import exposure,util
import matplotlib.pyplot as plt
Datagen_Train = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rescale=1./255,
        fill_mode='nearest',
        horizontal_flip=True,
        vertical_flip=True,
        brightness_range = [0.6,1.0],
        data_format=K.image_data_format()
        )
Datagen_Test = ImageDataGenerator(rescale=1./255,data_format=K.image_data_format())
print("Datagen Train & Test  Created!")

In [ ]:
train_generator = Datagen_Train.flow_from_directory(
        train_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        shuffle = True, 
        class_mode='categorical')

validation_generator = Datagen_Test.flow_from_directory(
        val_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        shuffle = True,
        class_mode='categorical')

test_generator = Datagen_Test.flow_from_directory(
        test_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        shuffle=False,
        class_mode='categorical')

In [ ]:
from matplotlib import pyplot 
x,y = train_generator.next()
Col = 10
Row = 10
# plot first few filters
n_filters, ix = 16, 1
fig = pyplot.figure(figsize=(18,18))
# plot each channel separately
for j in range(1,30):
  ix += 1
  # specify subplot and turn of axis
  ax = pyplot.subplot(Col, Row, ix)
  ax.set_xticks([])
  ax.set_yticks([])
  # plot filter channel in grayscale
  pyplot.imshow(x[ix, :, :, :],cmap="gray")
# show the figure
pyplot.show()

# Model_Spike

## Spike Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Activation, Dense, Flatten, Dropout, Conv2D, MaxPooling2D, Input, Concatenate,Add, BatchNormalization
from tensorflow.keras.initializers import glorot_uniform
inputs = Input(shape=(img_width,img_height,3))
X = Conv2D(filters=16,kernel_size=(3,3),padding="same", activation="relu")(inputs)
X = Conv2D(filters=32,kernel_size=(3,3),padding="same", activation="relu")(X)
X = MaxPooling2D(pool_size=(2,2),strides=(2,2))(X)
X = Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu")(X)
X = MaxPooling2D(pool_size=(2,2),strides=(2,2))(X)
X_shortcut = X

#Create Towers 
tower_1 = Conv2D(64, (1,1), padding='same', activation='relu')(X)
tower_1 = Conv2D(64, (3,3), padding='same', activation='relu')(tower_1)
tower_2 = Conv2D(64, (1,1), padding='same', activation='relu')(X)
tower_2 = Conv2D(64, (5,5), padding='same', activation='relu')(tower_2)
tower_3 = Conv2D(64, (1,1), padding='same', activation='relu')(X)
X = Concatenate(axis=3)([tower_1, tower_2, tower_3]) #Concatinate towers

# ResNet Skipconnections
X = Conv2D(filters = 64, kernel_size = (1, 1), strides = (1,1), padding = 'valid', kernel_initializer = glorot_uniform(seed=0))(X)
X = BatchNormalization(axis = 3)(X)
X = Activation('relu')(X)
X = Conv2D(filters = 64, kernel_size = (3, 3), strides = (1,1), padding = 'same', kernel_initializer = glorot_uniform(seed=0))(X)
X = BatchNormalization(axis = 3)(X)
X = Activation('relu')(X)
X = Conv2D(filters = 64, kernel_size = (1, 1), strides = (1,1), padding = 'valid', kernel_initializer = glorot_uniform(seed=0))(X)
X = BatchNormalization(axis = 3)(X)
X = Add()([X, X_shortcut ])
X = Activation('relu')(X)
X = MaxPooling2D(pool_size=(2,2),strides=(2,2))(X)

X = Flatten()(X)

predictions = Dense(units=no_of_classes, activation="softmax")(X)
model = Model(inputs=inputs, outputs=predictions)
print(model.summary())

## Compile

In [32]:
from tensorflow.keras import optimizers
adam = optimizers.Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])
print('Compiled!')

Compiled!


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
import math

Ca = ModelCheckpoint(filepath = 'Model_A.hdf5', monitor='val_accuracy', mode='max', verbose = 1, save_best_only = True)
Cl = ModelCheckpoint(filepath = 'Model_L.hdf5', monitor='val_loss', mode='min', verbose = 1, save_best_only = True)
early_stop = EarlyStopping(monitor='val_loss', patience=18)

history = model.fit(train_generator,
                    epochs=200, 
                    validation_data=validation_generator,  
                    callbacks = [Cl,Ca,early_stop],
                    validation_steps = cal_step(val_dir,batch_size),
                    steps_per_epoch = cal_step(train_dir,batch_size)
										)
print("Done")

In [ ]:
# Save and Load Model
model.load_weights("Model_L.hdf5")
model.save("model.h5")

In [ ]:
from tensorflow.keras.models import load_model
model = load_model("model.h5")
model.summary()

# Plot confusion matrix

In [ ]:
from numpy import argmax
import pandas as pd
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report
import seaborn as sn
import numpy as np
# model.load_weights("Model_L.hdf5")
class_labels = sorted(test_generator.class_indices.keys())
np.set_printoptions(precision=2)
y_prob = model.predict(test_generator, verbose=1,steps=cal_step(test_dir,batch_size)) 
predicted = np.argmax(y_prob,axis=1)
actual = test_generator.classes
actual = actual[:predicted.shape[0]]
print(predicted.shape)
print(actual.shape)
print(actual)
print(predicted)

results = confusion_matrix(actual, predicted)
acc = accuracy_score(actual, predicted) 
report = classification_report(actual, predicted)
print(results) 
print ('Accuracy Score :',acc) 
print ('Report : ', report)
result_json_file = 'confmat.txt' 
with open(result_json_file, mode='w') as f:
    print(results,file=f)
print("confmat.txt Done!!")

report_json_file = 'report.txt' 
with open(report_json_file, mode='w') as f:
  f.write(report)
print("report.txt Done!!")
report_df = pd.DataFrame(results) 
report_json_file = 'result.json' 
with open(report_json_file, mode='w') as f:
    report_df.to_json(f)
print("result.json Done!!")

sn.set(font_scale=1)
plt.figure(figsize = (no_of_classes,no_of_classes))
map = sn.heatmap(results, annot=True,annot_kws={"size": 20}, center=0,cmap="Blues", fmt='.1f',lw=0.5, cbar=True, cbar_kws={'label': '# Images', 'orientation': 'horizontal'})
map.set_title('Confusion matrix')
map.set_xticklabels(class_labels, fontsize = 5)
map.set_yticklabels(class_labels, fontsize = 5)
figure = map.get_figure()   
figure.savefig("output.png")

# Evaluate Model

In [ ]:
# evaluate and print test accuracy
model.load_weights("Model_L.hdf5")
score = model.evaluate(test_generator, verbose=1, steps = cal_step(test_dir,batch_size))
print('\n', 'Test accuracy:', score)
pred_json_file = 'predresult.txt' 
with open(pred_json_file, mode='w') as f:
    f.write("Loss: {}\nAcc: {}".format(score[0],score[1]))
print("predresult.txt Done!!")

# Save Model

In [ ]:
#Finally lets visualize the loss and accuracy wrt epochs

import matplotlib.pyplot as plt 
fig = plt.figure(figsize=(16, 16)) 
   
 # summarize history for accuracy  
   
plt.subplot(211)  
plt.plot(history.history['accuracy'])  
plt.plot(history.history['val_accuracy'])  
plt.title('model accuracy')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
plt.legend(['train', 'test'], loc='upper left')  
   
 # summarize history for loss  
plt.subplot(212)  
plt.plot(history.history['loss'])  
plt.plot(history.history['val_loss'])  
plt.title('model loss')  
plt.ylabel('loss')  
plt.xlabel('epoch')  
plt.legend(['train', 'test'], loc='upper left')  
plt.show()
fig.savefig("Plot")